In [1]:
import Python as py
import os
import pandas as pd

In [2]:

import os
import pandas as pd
root = os.getcwd()
df_scheduled = pd.read_csv(root + '/data/scheduled_loan_repayments.csv')
df_actual = pd.read_csv(root + '/data/actual_loan_repayments.csv')
df_balances = py.calculate_df_balances(df_scheduled,df_actual)

In [3]:
py.question_1(df_balances)

15.0

In [4]:
py.question_2(df_balances)

1.7

In [5]:
py.question_3(df_balances)

0.0

In [6]:
py.question_4(df_balances)

979573.17

In [7]:
##created
df_scheduled

,LoanID,LoanAmount,ScheduledRepayment
0,1,52000.0,2399.54
1,2,115000.0,5306.67
2,3,91000.0,4199.19
3,4,77000.0,3553.16
4,5,29000.0,1338.20
...,...,...,...
995,996,22000.0,1015.19
996,997,111000.0,5122.09
997,998,27000.0,1245.91
998,999,115000.0,5306.67


In [8]:
##created
df_actual

,RepaymentID,LoanID,Month,ActualRepayment
0,1.0,86,1,2168.81
1,2.0,17,1,2076.52
2,3.0,131,1,2630.26
3,4.0,345,1,2537.97
4,5.0,44,1,5075.94
...,...,...,...,...
11995,11996.0,797,12,5214.38
11996,11997.0,410,12,1015.19
11997,11998.0,202,12,0.00
11998,11999.0,607,12,3137.86


In [9]:
df_merged = pd.merge(df_actual,df_scheduled)
df_merged = df_merged.sort_values(['LoanID','Month']).reset_index().drop('index',axis=1)
df_merged

,RepaymentID,LoanID,Month,ActualRepayment,LoanAmount,ScheduledRepayment
0,544.0,1,1,2399.54,52000.0,2399.54
1,1313.0,1,2,2399.54,52000.0,2399.54
2,2551.0,1,3,2399.54,52000.0,2399.54
3,3636.0,1,4,2399.54,52000.0,2399.54
4,4300.0,1,5,2399.54,52000.0,2399.54
...,...,...,...,...,...,...
11995,7973.0,1000,8,2768.70,60000.0,2768.70
11996,8115.0,1000,9,2768.70,60000.0,2768.70
11997,9867.0,1000,10,2768.70,60000.0,2768.70
11998,10128.0,1000,11,2768.70,60000.0,2768.70


In [10]:
loans = df_merged[(df_merged['LoanID']==1 )|( df_merged['LoanID']==2)]
loan = df_merged[df_merged['LoanID']==1 ]

loans=loans.copy()
loan=loan.copy()

In [11]:
def calculate_columns(loan):
    r_pm = 0.1/12
    balance_start =[]
    interest_payment = []
    scheduled_principal =[]
    unscheduled_principal =[]
    
    for i, row in loan.iterrows():
        if  balance_start:
            balance_start.append(balance_end[-1])
            balance_end.append(balance_start[-1]*(1+r_pm) - row['ActualRepayment'])
        else:
            balance_start = [row['LoanAmount']]
            balance_end = [row['LoanAmount']*(1+r_pm)-row['ActualRepayment']]
    
        interest_payment.append(balance_start[-1]*r_pm)
        scheduled_principal.append(row['ScheduledRepayment']-balance_start[-1]*r_pm)   
        unscheduled_principal.append(row['ActualRepayment']-row['ScheduledRepayment'] if (row['ActualRepayment']>row['ScheduledRepayment']) else 0.0)

    loan['LoanBalanceStart'] = [round(n, 2) for n in balance_start]
    loan['LoanBalanceEnd']=[round(n, 2) for n in balance_end]
    loan['InterestPayment']=[round(n, 2) for n in interest_payment]
    loan['ScheduledPrincipal']=[round(n, 2) for n in scheduled_principal]
    loan['UnscheduledPrincipal']=[round(n, 2) for n in unscheduled_principal]
    return loan



In [12]:
df_balances = df_merged.groupby('LoanID').apply(calculate_columns, include_groups=False)
df_balances= df_balances .reset_index()[['RepaymentID', 'LoanID', 'Month', 'ActualRepayment','LoanAmount', 'ScheduledRepayment',
                                         'LoanBalanceStart','LoanBalanceEnd', 'InterestPayment', 'ScheduledPrincipal','UnscheduledPrincipal']]

In [13]:
df_balances

,RepaymentID,LoanID,Month,ActualRepayment,LoanAmount,ScheduledRepayment,LoanBalanceStart,LoanBalanceEnd,InterestPayment,ScheduledPrincipal,UnscheduledPrincipal
0,544.0,1,1,2399.54,52000.0,2399.54,52000.00,50033.79,433.33,1966.21,0.0
1,1313.0,1,2,2399.54,52000.0,2399.54,50033.79,48051.20,416.95,1982.59,0.0
2,2551.0,1,3,2399.54,52000.0,2399.54,48051.20,46052.09,400.43,1999.11,0.0
3,3636.0,1,4,2399.54,52000.0,2399.54,46052.09,44036.32,383.77,2015.77,0.0
4,4300.0,1,5,2399.54,52000.0,2399.54,44036.32,42003.74,366.97,2032.57,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11995,7973.0,1000,8,2768.70,60000.0,2768.70,43716.52,41312.12,364.30,2404.40,0.0
11996,8115.0,1000,9,2768.70,60000.0,2768.70,41312.12,38887.69,344.27,2424.43,0.0
11997,9867.0,1000,10,2768.70,60000.0,2768.70,38887.69,36443.05,324.06,2444.64,0.0
11998,10128.0,1000,11,2768.70,60000.0,2768.70,36443.05,33978.05,303.69,2465.01,0.0


In [14]:
calculate_columns(loan=loan)

,RepaymentID,LoanID,Month,ActualRepayment,LoanAmount,ScheduledRepayment,LoanBalanceStart,LoanBalanceEnd,InterestPayment,ScheduledPrincipal,UnscheduledPrincipal
0,544.0,1,1,2399.54,52000.0,2399.54,52000.00,50033.79,433.33,1966.21,0.0
1,1313.0,1,2,2399.54,52000.0,2399.54,50033.79,48051.20,416.95,1982.59,0.0
2,2551.0,1,3,2399.54,52000.0,2399.54,48051.20,46052.09,400.43,1999.11,0.0
3,3636.0,1,4,2399.54,52000.0,2399.54,46052.09,44036.32,383.77,2015.77,0.0
4,4300.0,1,5,2399.54,52000.0,2399.54,44036.32,42003.74,366.97,2032.57,0.0
5,5847.0,1,6,2399.54,52000.0,2399.54,42003.74,39954.24,350.03,2049.51,0.0
6,6362.0,1,7,2399.54,52000.0,2399.54,39954.24,37887.65,332.95,2066.59,0.0
7,7520.0,1,8,2399.54,52000.0,2399.54,37887.65,35803.84,315.73,2083.81,0.0
8,8699.0,1,9,2399.54,52000.0,2399.54,35803.84,33702.66,298.37,2101.17,0.0
9,9438.0,1,10,2399.54,52000.0,2399.54,33702.66,31583.98,280.86,2118.68,0.0


In [15]:
def calculate_df_balances(df_scheduled,df_actual):
    """ 
        This is a utility function that creates a merged dataframe that will be used in the following questions. 
        This function will not be graded directly.

        Args:
            df_scheduled (DataFrame): Dataframe created from the 'scheduled_loan_repayments.csv' dataset
            df_actual (DataFrame): Dataframe created from the 'actual_loan_repayments.csv' dataset
        
        Returns:
            DataFrame: A merged Dataframe 

            Columns after the merge should be: 
            ['RepaymentID', 'LoanID', 'Month', 'ActualRepayment', 'LoanAmount', 'ScheduledRepayment']

            Additional columns to be used in later questions should include: 
            ['UnscheduledPrincipal', 'LoanBalanceStart, 'LoanBalanceEnd'] 
            Note: 'LoanBalanceStart' for the first month of each loan should equal the 'LoanAmount'

            You may create other columns to assist you in your calculations. e.g:
            ['InterestPayment']

    """
    #merge the two dataframes together:
    df_merged = pd.merge(df_actual,df_scheduled)
    df_merged = df_merged.sort_values(['LoanID','Month']).reset_index().drop('index',axis=1) #Sort the columns using the loans and month columns.

    def calculate_columns(loan):
        r_pm = 0.1/12 #calculate the monly interest rate
        balance_start =[]
        interest_payment = []
        scheduled_principal =[]
        unscheduled_principal =[]
        
        for i, row in loan.iterrows():
            if  balance_start: #if the balance_start list is not empty this section of code will be executed.
                balance_start.append(balance_end[-1])
                balance_end.append(balance_start[-1]*(1+r_pm) - row['ActualRepayment'])
            else: #if the balance_start list is empty this section of code will be executed.
                balance_start = [row['LoanAmount']] #initialise the balance_start as the first element of the amount column.
                balance_end = [balance_start[-1]*(1+r_pm)-row['ActualRepayment']]
        
            interest_payment.append(balance_start[-1]*r_pm)
            scheduled_principal.append(row['ScheduledRepayment']-balance_start[-1]*r_pm)   
            unscheduled_principal.append(row['ActualRepayment']-row['ScheduledRepayment'] if (row['ActualRepayment']>row['ScheduledRepayment']) else 0.0)
        
        loan['LoanBalanceStart'] = [round(n, 2) for n in balance_start]
        loan['LoanBalanceEnd']=[round(n, 2) for n in balance_end]
        loan['InterestPayment']=[round(n, 2) for n in interest_payment]
        loan['ScheduledPrincipal']=[round(n, 2) for n in scheduled_principal]
        loan['UnscheduledPrincipal']=[round(n, 2) for n in unscheduled_principal]
        return loan

    df_balances = df_merged.groupby('LoanID').apply(calculate_columns, include_groups=False)
    df_balances= df_balances.reset_index()[['RepaymentID', 'LoanID', 'Month', 'ActualRepayment','LoanAmount', 'ScheduledRepayment',
                                         'LoanBalanceStart','LoanBalanceEnd', 'InterestPayment', 'ScheduledPrincipal','UnscheduledPrincipal']]
    return df_balances

In [16]:
calculate_df_balances(df_scheduled,df_actual)

,RepaymentID,LoanID,Month,ActualRepayment,LoanAmount,ScheduledRepayment,LoanBalanceStart,LoanBalanceEnd,InterestPayment,ScheduledPrincipal,UnscheduledPrincipal
0,544.0,1,1,2399.54,52000.0,2399.54,52000.00,50033.79,433.33,1966.21,0.0
1,1313.0,1,2,2399.54,52000.0,2399.54,50033.79,48051.20,416.95,1982.59,0.0
2,2551.0,1,3,2399.54,52000.0,2399.54,48051.20,46052.09,400.43,1999.11,0.0
3,3636.0,1,4,2399.54,52000.0,2399.54,46052.09,44036.32,383.77,2015.77,0.0
4,4300.0,1,5,2399.54,52000.0,2399.54,44036.32,42003.74,366.97,2032.57,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11995,7973.0,1000,8,2768.70,60000.0,2768.70,43716.52,41312.12,364.30,2404.40,0.0
11996,8115.0,1000,9,2768.70,60000.0,2768.70,41312.12,38887.69,344.27,2424.43,0.0
11997,9867.0,1000,10,2768.70,60000.0,2768.70,38887.69,36443.05,324.06,2444.64,0.0
11998,10128.0,1000,11,2768.70,60000.0,2768.70,36443.05,33978.05,303.69,2465.01,0.0


In [17]:
import numpy as np

calculate_df_balances(df_scheduled,df_actual)

,RepaymentID,LoanID,Month,ActualRepayment,LoanAmount,ScheduledRepayment,LoanBalanceStart,LoanBalanceEnd,InterestPayment,ScheduledPrincipal,UnscheduledPrincipal
0,544.0,1,1,2399.54,52000.0,2399.54,52000.00,50033.79,433.33,1966.21,0.0
1,1313.0,1,2,2399.54,52000.0,2399.54,50033.79,48051.20,416.95,1982.59,0.0
2,2551.0,1,3,2399.54,52000.0,2399.54,48051.20,46052.09,400.43,1999.11,0.0
3,3636.0,1,4,2399.54,52000.0,2399.54,46052.09,44036.32,383.77,2015.77,0.0
4,4300.0,1,5,2399.54,52000.0,2399.54,44036.32,42003.74,366.97,2032.57,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11995,7973.0,1000,8,2768.70,60000.0,2768.70,43716.52,41312.12,364.30,2404.40,0.0
11996,8115.0,1000,9,2768.70,60000.0,2768.70,41312.12,38887.69,344.27,2424.43,0.0
11997,9867.0,1000,10,2768.70,60000.0,2768.70,38887.69,36443.05,324.06,2444.64,0.0
11998,10128.0,1000,11,2768.70,60000.0,2768.70,36443.05,33978.05,303.69,2465.01,0.0


In [18]:
 #     Calculate the percent of loans that defaulted as per the type 1 default definition 
        
    #     Args:
    #         df_balances (DataFrame): Dataframe created from the 'calculate_df_balances()' function
        
    #     Returns:
    #         float: The percentage of defaulted loans (type 1)
    
# A type 1 default will occur on a loan when a repayment is missed.

In [19]:
def question_1(df_balances):
    #missed payments are denoted by a 0 in the ActualRepayment column.
    loans_missing_payments = len(df_balances[df_balances['ActualRepayment']==0]['LoanID'].unique())
    
    #Calculate the total amount of loans in the dataset:
    total_loans = len(df_balances['LoanID'].unique())
    
    default_rate_percent = loans_missing_payments/total_loans*100
    return default_rate_percent

In [20]:
# pip install matplotlib

In [21]:
question_1(df_balances=df_balances)

15.0

# Question 2

In [22]:
def question_2(df_balances, df_scheduled = None):
    """ 
    A type 2 default will occur on a loan when more than 15% of the expected total payments are unpaid for the year.

        Calculate the percent of loans that defaulted as per the type 2 default definition 
        
        Args:
            df_balances (DataFrame): Dataframe created from the 'calculate_df_balances()' function
            df_scheduled (DataFrame): Dataframe created from the 'scheduled_loan_repayments.csv' dataset
        
        Returns:
            float: The percentage of defaulted loans (type 2)

    """
    #Note that only the df_balances input dataframes will be used.
    T2_margin = 12*0.15 #The number of missed payments which result in a T2 default if exceeded.

    #Calculate the number of missed payments per loan.
    missed_payments_per_loan = df_balances[df_balances['ActualRepayment']==0].groupby('LoanID').count()
    
    #Count the number of loans which has more missed payments than the allowable margin.
    T2_default_number = sum(missed_payments_per_loan['Month'] > T2_margin)
    
    #Calculate the total amount of loans in the dataset:
    total_loans = len(df_balances['LoanID'].unique())
    
    default_rate_percent = 100*T2_default_number/total_loans
    return default_rate_percent

In [23]:
question_2(df_scheduled=df_scheduled, df_balances=df_balances)

1.7

In [24]:
# #Sum up all the payments made during the year of each individual loan.
# scheduled_payments = df_balances.groupby('LoanID').sum()['ScheduledRepayment']
# actual_payments = df_balances.groupby('LoanID').sum()['ActualRepayment']

# #calculate the number of loans where more than 15% of the expected paym
# T2_default_number = sum((((scheduled_payments- actual_payments)/scheduled_payments)*100) >15 )

# #Calculate the total amount of loans in the dataset:
# total_loans = len(df_balances['LoanID'].unique())


In [25]:
#Sum up all the payments made during the year of each individual loan.
scheduled_payments = df_balances.groupby('LoanID').sum()['ScheduledRepayment']
actual_payments = df_balances.groupby('LoanID').sum()['ActualRepayment']

#calculate the number of loans where more than 15% of the expected paym
T2_default_number = sum((((scheduled_payments- actual_payments)/scheduled_payments)*100) >15 )

#Calculate the total amount of loans in the dataset:
total_loans = len(df_balances['LoanID'].unique())

In [26]:
T2_margin = 12*0.15 #The number of missed payments which result in a T2 default if exceeded.

#Calculate the number of missed payments per loan.
missed_payments_per_loan = df_balances[df_balances['ActualRepayment']==0].groupby('LoanID').count()

#Count the number of loans which has more missed payments than the allowable margin.
T2_default_number = sum(missed_payments_per_loan['Month'] > miss_margin)

#Calculate the total amount of loans in the dataset:
total_loans = len(df_balances['LoanID'].unique())

(T2_default_number/total_loans)*100

NameError: name 'miss_margin' is not defined

In [ ]:
df_balances[df_balances['ActualRepayment']==0].groupby('LoanID').count()['Month'].plot()

# Question 3

In [ ]:
def question_3(df_balances):
    """ 
        Calculate the anualized CPR (As a %) from the geometric mean SMM.
        SMM is calculated as: (Unscheduled Principal)/(Start of Month Loan Balance)
        CPR is calcualted as: 1 - (1- SMM_mean)^12  

        Args:
            df_balances (DataFrame): Dataframe created from the 'calculate_df_balances()' function

        Returns:
            float: The anualized CPR of the loan portfolio as a percent.
            
    """
    #Create a SMM column which conatins the Single Monthly Mortality Rate for every month of each loan.
    df_balances['SMM'] = df_balances['UnscheduledPrincipal']/df_balances['LoanBalanceStart']

    def geometric_mean(column):
        """" Calculate the geometric mean of a input column
        Args: 
            column (Pandas Series) : Any series containing numeric values.
        
        Returns:
            float: The geometric mean of the input series."""
        prod = 1
        for i in column:
            prod *= i
        return prod**(1/len(column))

    #Calculate the cpr_percent for the entire loan portfolio using the supplied equation [CPR = 1 - (1- SMM_mean)^12].
    cpr_percent = (1-(1-geometric_mean(df_balances['SMM']))**12)*100

    # The Geometric mean used to calculate the mean of the SMM results in a value of 0 when the series contains one or more 0 value. Since the series 
    # contains many zeros due to unsheduled payments being scarce, arithmetic mean seams like a more appropriate solution. The followibng line of code
    # can be uncommented to use the arithmetic mean which results in an annualized CPR percentage of 3.64%. However since the question specifically asks for 
    # geometic mean the CPR value of 0 calculated using the geometric mean is returned by the function.
    # cpr_percent = (1-(1-df_balances['SMM'].mean())**12)*100
    return cpr_percent

In [ ]:
question_3(df_balances=df_balances)

In [27]:
unscheduled_principal_per_loan = df_balances.groupby('LoanID').sum()['UnscheduledPrincipal']

In [28]:
type(df_balances['SMM'])

KeyError: 'SMM'

In [ ]:
df_balances['SMM'] = df_balances['UnscheduledPrincipal']/df_balances['LoanBalanceStart']

In [ ]:
df_balances['SMM']

In [29]:
# def geometric_mean(loan):
#     prod = 1
#     for i in loan['SMM']:
#         prod *= i
#     SMM_mean = loan['SMM'].mean()
#     cpr = 1-(1-SMM_mean)**12
    
#     loan['mean_SMM']=SMM_mean
#     return loan

def geometric_mean(column):
    prod = 1
    for i in column:
        prod *= i
    geo_mean = prod**(1/len(column))
    return geo_mean

In [30]:
df_balances['SMM'].mean()

KeyError: 'SMM'

In [31]:
geometric_mean(df_balances['SMM'])

KeyError: 'SMM'

In [32]:
# df_balances[['LoanID','SMM']].groupby('LoanID').apply(geometric_mean)

In [33]:
for i in df_balances['LoanID'].unique():
    df = df_balances[df_balances['LoanID']==i]
    

# Question 4

In [40]:
def question_4(df_balances):
    """ 
        Calculate the predicted total loss for the second year in the loan term.
        Use the equation: probability_of_default * total_loan_balance * (1 - recovery_rate).
        The probability_of_default value must be taken from either your question_1 or question_2 answer. 
        Decide between the two answers based on which default definition you believe to be the more useful metric.
        Assume a recovery rate of 80% 
        
        Args:
            df_balances (DataFrame): Dataframe created from the 'calculate_df_balances()' function
        
        Returns:
            float: The predicted total loss for the second year in the loan term.
            
    """
    probability_of_default = question_1(df_balances=df_balances)/100 # divide by 100 to get probability.
    recovery_rate = 0.8
    
    #The total loan balance is taken as the sum of all the outstanding loan balances which is equavalent to the sum of the loan end balances' of the 12th month of the loan.
    total_loan_balance = sum(df_balances[df_balances['Month']== 12]['LoanBalanceEnd'])
    
    total_loss = round(probability_of_default*total_loan_balance*(1-recovery_rate),2)
    return total_loss

In [41]:
probability_of_default = question_2(df_balances=df_balances)/100 # divide by 100 to get probability.
recovery_rate = 0.8

#The total loan balance is taken as the sum of all the outstanding loan balances which is equavalent to the sum of the loan end balances' of the 12th month of the loan.
total_loan_balance = sum(df_balances[df_balances['Month']== 12]['LoanBalanceEnd'])

totoal_loss = probability_of_default*total_loan_balance*(1-recovery_rate)

In [42]:
sum(df_balances.groupby('LoanID').mean()['LoanAmount'])

64950000.0

In [43]:
question_4(df_balances)

979573.17

In [38]:
probability_of_default

0.017

In [39]:
df_balances

,RepaymentID,LoanID,Month,ActualRepayment,LoanAmount,ScheduledRepayment,LoanBalanceStart,LoanBalanceEnd,InterestPayment,ScheduledPrincipal,UnscheduledPrincipal
0,544.0,1,1,2399.54,52000.0,2399.54,52000.00,50033.79,433.33,1966.21,0.0
1,1313.0,1,2,2399.54,52000.0,2399.54,50033.79,48051.20,416.95,1982.59,0.0
2,2551.0,1,3,2399.54,52000.0,2399.54,48051.20,46052.09,400.43,1999.11,0.0
3,3636.0,1,4,2399.54,52000.0,2399.54,46052.09,44036.32,383.77,2015.77,0.0
4,4300.0,1,5,2399.54,52000.0,2399.54,44036.32,42003.74,366.97,2032.57,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11995,7973.0,1000,8,2768.70,60000.0,2768.70,43716.52,41312.12,364.30,2404.40,0.0
11996,8115.0,1000,9,2768.70,60000.0,2768.70,41312.12,38887.69,344.27,2424.43,0.0
11997,9867.0,1000,10,2768.70,60000.0,2768.70,38887.69,36443.05,324.06,2444.64,0.0
11998,10128.0,1000,11,2768.70,60000.0,2768.70,36443.05,33978.05,303.69,2465.01,0.0
